## Momentum Trading Strategy Backtest

**Summary of project:**

### Libraries 

In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import quantstats as qs
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## Importing ETFs 

**Why ETFS?**
> Momentum can look bad on single stocks because a lot of the price move is idiosyncratic noise like one earnings surprise, one headline, one downgrade. An ETF averages many stocks, so that idiosyncratic component diversifies away. What is left is more systematic trend, sector, factor and macro drift, which is exactly what momentum is trying to capture.

In [ ]:
# 1) Define the ETF universe you chose
tickers = ["VTI", "QQQ", "IWM", "VEA", "VWO", "BND", "TLT", "GLD"]

# 2) Download daily adjusted prices
prices = yf.download(
    tickers,
    start="2015-01-01",
    end="2026-01-01",
    interval="1d",
# auto_adjust=True returns prices adjusted for splits and dividends (better for returns)
    auto_adjust=True,     
    progress=False
)["Close"]

# 3) cleaning: sort by date, drop days where everything is missing
prices = prices.sort_index()
prices = prices.dropna(how="all")

# 4) Compute simple daily returns
returns = prices.pct_change().dropna()

# 5) Quick sanity checks
print("Prices shape:", prices.shape)
print("Returns shape:", returns.shape)
print("\nLast 5 rows of prices:\n", prices.tail())
print("\nLast 5 rows of returns:\n", returns.tail())

